### Analysis detail:

In this section we take the below csv files created in other analysis and combine them into one file which is then used to create a heatmap that shows the average income by postcode.

The csv's used include:
* Vic suburbs median house prices for the past 10 years (with postcodes added for each suburb)
* The Vic income by postcode
* The Vic suburbs latitude and Longitude

There is a second part of the analysis that looks at the relationship between suburb house price and average income for the year of 2018.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
import requests
import json
import gmaps
import os

# Importing Google maps API key
from api_keys import g_key

# Importing all the csv files we need to merge.
prices_path = "Data files/prices with postcode.csv"
salary_path = "Data files/vic_wages_postcode.csv"
lat_lng_path = "Data files/subs lat and lng.csv"

# Reading in the files.
prices_df = pd.read_csv(prices_path)
salary_df = pd.read_csv(salary_path)
lat_lng_df = pd.read_csv(lat_lng_path)

In [2]:
# Merging the first 2 csv files.
combined_data = pd.merge(prices_df, salary_df, on="Postcode", how="left")

In [3]:
# Displaying the dataframe.
combined_data

,Suburb,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Postcode,Average income
0,ABBOTSFORD,736000.0,730000.0,714000.0,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1030000.0,1150000.0,3067,71646.487443
1,ABERFELDIE,1046500.0,994000.0,852500.0,947500.0,1045000.0,1207500.0,1300000.0,1453000.0,1500000.0,1390000.0,1592500.0,3040,78506.787179
2,AIREYS INLET,606000.0,680000.0,634000.0,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1005000.0,3231,57331.725581
3,AIRPORT WEST,575000.0,557500.0,495000.0,532000.0,575000.0,635000.0,742000.0,845000.0,847500.0,790000.0,900000.0,3042,61603.532710
4,ALBANVALE,320000.0,317000.0,310000.0,313000.0,326500.0,345000.0,439000.0,526500.0,546000.0,512000.0,563000.0,3021,42223.782280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,YARRAM,166000.0,195500.0,215000.0,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,3971,42120.961846
658,YARRAVILLE,647000.0,607000.0,613000.0,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,950000.0,3013,76176.411410
659,YARRAWONGA,297000.0,270000.0,300000.0,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,398000.0,3730,46917.999593
660,YEA,240000.0,270000.0,255000.0,260000.0,255000.0,260000.0,305000.0,386500.0,392500.0,435500.0,416500.0,3717,46668.789632


In [4]:
# Merging the 3rd csv file.
combined_data = pd.merge(combined_data, lat_lng_df, on="Suburb", how="left")

In [5]:
# Displaying the dataframe.
combined_data

,Suburb,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Postcode,Average income,Lat,Lng
0,ABBOTSFORD,736000.0,730000.0,714000.0,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1030000.0,1150000.0,3067,71646.487443,-37.803000,145.002000
1,ABERFELDIE,1046500.0,994000.0,852500.0,947500.0,1045000.0,1207500.0,1300000.0,1453000.0,1500000.0,1390000.0,1592500.0,3040,78506.787179,-37.762000,144.901000
2,AIREYS INLET,606000.0,680000.0,634000.0,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1005000.0,3231,57331.725581,-38.461511,144.105766
3,AIRPORT WEST,575000.0,557500.0,495000.0,532000.0,575000.0,635000.0,742000.0,845000.0,847500.0,790000.0,900000.0,3042,61603.532710,-37.726234,144.881353
4,ALBANVALE,320000.0,317000.0,310000.0,313000.0,326500.0,345000.0,439000.0,526500.0,546000.0,512000.0,563000.0,3021,42223.782280,-37.746000,144.765000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,YARRAM,166000.0,195500.0,215000.0,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,3971,42120.961846,-38.565594,146.675105
658,YARRAVILLE,647000.0,607000.0,613000.0,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,950000.0,3013,76176.411410,-37.820274,144.882170
659,YARRAWONGA,297000.0,270000.0,300000.0,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,398000.0,3730,46917.999593,-36.027113,145.999058
660,YEA,240000.0,270000.0,255000.0,260000.0,255000.0,260000.0,305000.0,386500.0,392500.0,435500.0,416500.0,3717,46668.789632,-37.222937,145.407020


In [6]:
# Dropping any rows with missing data.
final_data = combined_data.dropna()
final_data

,Suburb,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,Postcode,Average income,Lat,Lng
0,ABBOTSFORD,736000.0,730000.0,714000.0,792500.0,862500.0,925000.0,1187500.0,1280000.0,1192500.0,1030000.0,1150000.0,3067,71646.487443,-37.803000,145.002000
1,ABERFELDIE,1046500.0,994000.0,852500.0,947500.0,1045000.0,1207500.0,1300000.0,1453000.0,1500000.0,1390000.0,1592500.0,3040,78506.787179,-37.762000,144.901000
2,AIREYS INLET,606000.0,680000.0,634000.0,664000.0,625500.0,680000.0,715000.0,737500.0,869000.0,985000.0,1005000.0,3231,57331.725581,-38.461511,144.105766
3,AIRPORT WEST,575000.0,557500.0,495000.0,532000.0,575000.0,635000.0,742000.0,845000.0,847500.0,790000.0,900000.0,3042,61603.532710,-37.726234,144.881353
4,ALBANVALE,320000.0,317000.0,310000.0,313000.0,326500.0,345000.0,439000.0,526500.0,546000.0,512000.0,563000.0,3021,42223.782280,-37.746000,144.765000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,YARRAM,166000.0,195500.0,215000.0,190000.0,202500.0,245000.0,230000.0,225000.0,218000.0,240000.0,260000.0,3971,42120.961846,-38.565594,146.675105
658,YARRAVILLE,647000.0,607000.0,613000.0,696000.0,710000.0,810000.0,900500.0,1000000.0,1045000.0,971000.0,950000.0,3013,76176.411410,-37.820274,144.882170
659,YARRAWONGA,297000.0,270000.0,300000.0,302500.0,297000.0,300500.0,320000.0,324500.0,345000.0,375000.0,398000.0,3730,46917.999593,-36.027113,145.999058
660,YEA,240000.0,270000.0,255000.0,260000.0,255000.0,260000.0,305000.0,386500.0,392500.0,435500.0,416500.0,3717,46668.789632,-37.222937,145.407020


In [7]:
# Saving the merged df's to a new csv file.
final_data.to_csv("Data files/median_house_price_income.csv", encoding="utf-8", index=False, header=True)

In [8]:
# Using the income data to generate a heatmap.
gmaps.configure(api_key=g_key)

locations = final_data[["Lat", "Lng"]]

salary = final_data["Average income"]/10000

fig = gmaps.figure(zoom_level=8, center=(-38, 145))

heat_layer = gmaps.heatmap_layer(locations, weights=salary, dissipating=False, max_intensity=10, point_radius=0.05)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

**Analysis Part 2:** Suburb and Income correlation
Unfortunately we were only able to find 1 year of data broken down by suburb.
This data was for 2018 and therefore below analysis is based on the house pricing in the year 2018.

In [14]:
# Start by getting the relevant columns
income_sub = final_data.loc[:, ["Suburb", "2018", "Average income"]]
income_sub

,Suburb,2018,Average income
0,ABBOTSFORD,1192500.0,71646.487443
1,ABERFELDIE,1500000.0,78506.787179
2,AIREYS INLET,869000.0,57331.725581
3,AIRPORT WEST,847500.0,61603.532710
4,ALBANVALE,546000.0,42223.782280
...,...,...,...
657,YARRAM,218000.0,42120.961846
658,YARRAVILLE,1045000.0,76176.411410
659,YARRAWONGA,345000.0,46917.999593
660,YEA,392500.0,46668.789632


In [20]:
# Next, we ranked the price column and income column from highest to lowest.
income_sub["Price Rank"] = income_sub["2018"].rank(method="max", ascending=False)
income_sub["Income Rank"] = income_sub["Average income"].rank(method="max", ascending=False)
income_sub

,Suburb,2018,Average income,Price Rank,Income Rank,Rank difference
435,NOOJEE,255000.0,36627.375691,572.0,647.0,78.0
322,KOONDROOK,268000.0,36913.835355,559.0,646.0,87.0
444,NYAH WEST,135000.0,39043.768212,628.0,645.0,17.0
472,PORT WELSHPOOL,295000.0,39105.138298,541.0,644.0,105.0
170,DALLAS,490000.0,39113.321639,397.0,643.0,247.0
...,...,...,...,...,...,...
84,BRIGHTON,2879000.0,137485.192246,2.0,5.0,3.0
378,MIDDLE PARK,2650000.0,142702.304610,5.0,4.0,-2.0
5,ALBERT PARK,2070000.0,142702.304610,15.0,4.0,-12.0
198,EAST MELBOURNE,2675000.0,185437.162308,3.0,2.0,-1.0


In [21]:
# Then we calculated the difference in the ranking
income_sub["Rank difference"] = income_sub["Price Rank"] - income_sub["Income Rank"]
income_sub

,Suburb,2018,Average income,Price Rank,Income Rank,Rank difference
435,NOOJEE,255000.0,36627.375691,572.0,647.0,-75.0
322,KOONDROOK,268000.0,36913.835355,559.0,646.0,-87.0
444,NYAH WEST,135000.0,39043.768212,628.0,645.0,-17.0
472,PORT WELSHPOOL,295000.0,39105.138298,541.0,644.0,-103.0
170,DALLAS,490000.0,39113.321639,397.0,643.0,-246.0
...,...,...,...,...,...,...
84,BRIGHTON,2879000.0,137485.192246,2.0,5.0,-3.0
378,MIDDLE PARK,2650000.0,142702.304610,5.0,4.0,1.0
5,ALBERT PARK,2070000.0,142702.304610,15.0,4.0,11.0
198,EAST MELBOURNE,2675000.0,185437.162308,3.0,2.0,1.0


In [27]:
# Creating a final rank to get an indication of suburbs with the biggest difference.
income_sub["Final Rank"] = ((income_sub["Price Rank"] + income_sub["Income Rank"]) / 2).round(0)
income_sub

,Suburb,2018,Average income,Price Rank,Income Rank,Rank difference,Final Rank
591,TOORAK,4700000.0,196816.360309,1.0,1.0,0.0,1.0
198,EAST MELBOURNE,2675000.0,185437.162308,3.0,2.0,1.0,2.0
378,MIDDLE PARK,2650000.0,142702.304610,5.0,4.0,1.0,4.0
5,ALBERT PARK,2070000.0,142702.304610,15.0,4.0,11.0,10.0
84,BRIGHTON,2879000.0,137485.192246,2.0,5.0,-3.0,4.0
...,...,...,...,...,...,...,...
170,DALLAS,490000.0,39113.321639,397.0,643.0,-246.0,520.0
472,PORT WELSHPOOL,295000.0,39105.138298,541.0,644.0,-103.0,592.0
444,NYAH WEST,135000.0,39043.768212,628.0,645.0,-17.0,636.0
322,KOONDROOK,268000.0,36913.835355,559.0,646.0,-87.0,602.0


In [28]:
# Sorting by final rank
income_sub = income_sub.sort_values("Final Rank")
income_sub

,Suburb,2018,Average income,Price Rank,Income Rank,Rank difference,Final Rank
591,TOORAK,4700000.0,196816.360309,1.0,1.0,0.0,1.0
198,EAST MELBOURNE,2675000.0,185437.162308,3.0,2.0,1.0,2.0
378,MIDDLE PARK,2650000.0,142702.304610,5.0,4.0,1.0,4.0
84,BRIGHTON,2879000.0,137485.192246,2.0,5.0,-3.0,4.0
357,MALVERN,2661000.0,128720.090503,4.0,6.0,-2.0,5.0
...,...,...,...,...,...,...,...
622,WEDDERBURN,130000.0,40240.197347,631.0,634.0,-3.0,632.0
184,DONALD,145000.0,39539.080471,627.0,640.0,-13.0,634.0
180,DERRINALLUM,122500.0,40087.212014,634.0,636.0,-2.0,635.0
444,NYAH WEST,135000.0,39043.768212,628.0,645.0,-17.0,636.0


In [29]:
# Viewing the top 10 results.
income_sub.head(10)

,Suburb,2018,Average income,Price Rank,Income Rank,Rank difference,Final Rank
591,TOORAK,4700000.0,196816.360309,1.0,1.0,0.0,1.0
198,EAST MELBOURNE,2675000.0,185437.162308,3.0,2.0,1.0,2.0
378,MIDDLE PARK,2650000.0,142702.304610,5.0,4.0,1.0,4.0
84,BRIGHTON,2879000.0,137485.192246,2.0,5.0,-3.0,4.0
357,MALVERN,2661000.0,128720.090503,4.0,6.0,-2.0,5.0
108,CANTERBURY,2536500.0,127984.906457,6.0,7.0,-1.0,6.0
259,HAWTHORN,2488000.0,106270.523211,7.0,11.0,-4.0,9.0
5,ALBERT PARK,2070000.0,142702.304610,15.0,4.0,11.0,10.0
476,PORTSEA,2180000.0,115369.548936,12.0,9.0,3.0,10.0
17,ARMADALE,2341500.0,106775.496305,9.0,10.0,-1.0,10.0


In [32]:
# Viewing the bottom 10 results.
income_sub.tail(10)

,Suburb,2018,Average income,Price Rank,Income Rank,Rank difference,Final Rank
307,KERANG,177000.0,42068.764563,614.0,618.0,-4.0,616.0
125,CHARLTON,150000.0,42103.633557,624.0,617.0,7.0,620.0
550,ST ARNAUD,145000.0,41970.416542,627.0,620.0,7.0,624.0
554,STANHOPE,161500.0,41085.943137,622.0,626.0,-4.0,624.0
650,WYCHEPROOF,124500.0,41581.983568,633.0,622.0,11.0,628.0
622,WEDDERBURN,130000.0,40240.197347,631.0,634.0,-3.0,632.0
184,DONALD,145000.0,39539.080471,627.0,640.0,-13.0,634.0
180,DERRINALLUM,122500.0,40087.212014,634.0,636.0,-2.0,635.0
444,NYAH WEST,135000.0,39043.768212,628.0,645.0,-17.0,636.0
480,PYRAMID HILL,100000.0,40275.995546,642.0,632.0,10.0,637.0
